In [1]:
!pip install fastparquet

In [2]:
import pandas as pd
import os
import re

In [9]:
def sort_num(el):
    num = re.search(r"version_02_(\w+)_(.+)\..+", el)
    part = num.group(2)
    if part == 'final':
        return float('inf')
    else:
        return int(part)
FOLDERS = ['init_df/', 'init_df_not_in_koi/']
START_INDEX = 0
END_INDEX = 2

In [10]:
'init_df' in '/kaggle/input/nasa-cooked/init_df_not_in_koi/'.split('/')

False

In [11]:
parquets = []
csvs = []
for f in FOLDERS:
    l = os.listdir(f)
    print(l)
    parquets.append([el for el in l if el.endswith('.parquet')])
    csvs.append([el for el in l if el.endswith('.csv')])
    
    parquets[-1].sort(key=sort_num)
    csvs[-1].sort(key=sort_num)

['df_version_02_value_14.parquet', 'df_version_02_id_13.csv', 'df_version_02_value_12.parquet', 'df_version_02_value_5.parquet', 'df_version_02_id_5.csv', 'df_version_02_id_7.csv', 'df_version_02_id_9.csv', 'df_version_02_value_9.parquet', 'df_version_02_id_10.csv', 'df_version_02_value_6.parquet', 'df_version_02_value_4.parquet', 'df_version_02_value_3.parquet', 'df_version_02_id_6.csv', 'df_version_02_value_15.parquet', 'df_version_02_id_15.csv', 'df_version_02_id_4.csv', 'df_version_02_id_2.csv', 'df_version_02_id_12.csv', 'df_version_02_id_11.csv', 'df_version_02_id_3.csv', 'df_version_02_id_8.csv', 'df_version_02_value_8.parquet', 'df_version_02_value_13.parquet', 'df_version_02_value_10.parquet', 'df_version_02_value_1.parquet', 'df_version_02_id_16.csv', 'df_version_02_value_11.parquet', 'df_version_02_id_14.csv', 'df_version_02_value_2.parquet', 'df_version_02_id_1.csv', 'df_version_02_value_7.parquet', 'df_version_02_value_16.parquet']
['df_version_02_value_14.parquet', 'df_ve

In [ ]:
# df_values_loaded = pd.read_parquet(FOLDERS[0] + 'df_version_02_value_1.parquet', engine="fastparquet")
# df_ids_loaded = pd.read_csv(FOLDERS[0] + 'df_version_02_id_1.csv')
# df_trunk = pd.concat([df_ids_loaded, df_values_loaded.T.reset_index(drop=True)], axis=1)
# df_trunk.set_index(['KEPID','PLANET_NUM'], inplace=True)
# full_df = pd.DataFrame(columns=df_trunk.columns)
# full_df.index.name = df_trunk.index.name

In [ ]:
dfs_to_concat = []
for f, csv_list, parquet_list in zip(FOLDERS, csvs, parquets):
    for pair in zip(csv_list[START_INDEX:END_INDEX], parquet_list[START_INDEX:END_INDEX]):
        df_values_loaded = pd.read_parquet(f + pair[1], engine="fastparquet")
        df_ids_loaded = pd.read_csv(f + pair[0])
        if 'init_df' in f.split('/'):
            df_part = pd.concat([df_ids_loaded, df_values_loaded.T.reset_index(drop=True)], axis=1)
        else:
            df_part = pd.concat([df_ids_loaded, df_values_loaded], axis=1)
        df_part.set_index(['KEPID', 'PLANET_NUM'], inplace=True)
        dfs_to_concat.append(df_part)

full_df = pd.concat(dfs_to_concat, axis=0)

In [13]:
df_trunk.tail()

,,LABEL,0,1,2,3,4,5,6,7,8,...,71990,71991,71992,71993,71994,71995,71996,71997,71998,71999
KEPID,PLANET_NUM,,,,,,,,,,,,,,,,,,,,,
4947726,1,2,0.001355,0.001094,0.000984,0.002461,0.001053,0.001571,0.000881,0.001698,0.000881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4948863,1,2,NaN,-0.000526,0.000629,-0.000221,-0.000096,0.000359,-0.000280,0.000072,-0.000017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8308516,2,1,0.000144,0.000246,0.000114,0.000394,-0.000027,0.000200,-0.000162,0.000184,-0.000020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5529501,1,2,0.000008,-0.000086,0.000228,0.000089,0.000073,0.000105,0.000106,0.000033,-0.000058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5529560,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_trunk['LABEL'].value_counts()

LABEL
2    3551
0    2688
1    1261
Name: count, dtype: int64

In [18]:
#{"CONFIRMED": 0, "CANDIDATE": 1, "FALSE POSITIVE": 2}
# marked candidates as confirmed to simulate robovetter. may change in the future
#df_trunk.loc[df_trunk['LABEL'] == 1, 'LABEL'] = 0

In [21]:
df_trunk_new = df_trunk.copy()

In [29]:
drop_df = pd.read_csv('../drop_df.csv').set_index(['kepid', 'koi_tce_plnt_num'])

In [30]:
drop_df.head()

,,koi_disposition,koi_score
kepid,koi_tce_plnt_num,,
10811496,1.000000,CANDIDATE,0.0
4936180,1.000000,CANDIDATE,0.0
5115978,1.243654,CANDIDATE,0.0
6599919,1.000000,CANDIDATE,0.0
7031517,1.000000,CANDIDATE,0.0


In [31]:
df_trunk_new.loc[(df_trunk_new['LABEL'] == 1) & (df_trunk_new.index.isin(drop_df.index)), 'LABEL'] = 2 #if it's candidate and present in drop_df means it's false positive
df_trunk_new.loc[(df_trunk_new['LABEL'] == 1) & (~df_trunk_new.index.isin(drop_df.index)), 'LABEL'] = 0 #if it's candidate and NOT present means it's prob a planet

In [33]:
df_trunk_new['LABEL'].value_counts()

LABEL
0    5552
2    2948
Name: count, dtype: int64

In [34]:
df_trunk['LABEL'].value_counts()

LABEL
0    3108
2    2755
1    2637
Name: count, dtype: int64

In [20]:
df_trunk.shape

(8500, 70001)